In [ ]:
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import LeaveOneOut, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import sklearn
 
def corregir_dtypes_dataframe(df):
    df.cantidad = pd.to_numeric(df.cantidad)
    return df

def crear_dataframe_desde_mongodb():
    cadena_conexion = ('mongodb://{nom_usuario}:{password}@{host}:{port}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'PY01_c02',
        password = 'P4rd83XkXrTz',
        host = '5.189.129.12',
        port = 27017
    )
    client = pymongo.MongoClient(cadena_conexion)
    db = client.PY01
    header = [*db.PY01.find_one().keys()]
    lista_dataset = list()
    for doc in db.PY01.find():
        lista_dataset.append([*doc.values()])
 
    df = pd.DataFrame(data = lista_dataset, columns = header)
    return df
 
def crear_codigos_prod(serie):
    prods = serie.unique()
    cantidad_prods = len(prods)
    prods = sorted(prods)
    return dict(zip(prods, range(cantidad_prods)))
 
def limpiar_dataset(df):
    dias = {'LUNES': 1, 'MARTES': 2, 'MIERCOLES': 3, 'JUEVES': 4, 'VIERNES': 5, 'SABADO': 6, 'SUNDAY': 7, 'DOMINGO': 7}
    meses = {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Setiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}
    for dia, cod in dias.items():
        df.loc[df.dia == dia, 'dia'] = cod
    df.dia = pd.to_numeric(df.dia)
    
    for mes, cod in meses.items():
        df.loc[df.mes == mes, 'mes'] = cod
    df.mes = pd.to_numeric(df.mes)
    
    for prod, cod in crear_codigos_prod(df['prod']).items():
        df.loc[df['prod'] == prod, 'prod'] = cod
    return df

def generar_variables(df):
    nuevos_registros = list()
    for row in df.itertuples():
        cantidad = row.cantidad
        if cantidad > 1:
            cantidad -= 1
            for t in range(cantidad):
                nuevos_registros.append(row)

    df = df.append(nuevos_registros, ignore_index = True).drop(labels = ['Index','cantidad'], axis = 1)

    df = pd.concat((df, pd.get_dummies(df.dia, prefix = 'dia')), axis = 1)
    df = df.assign(dia_sin = lambda x: np.sin(2 * np.pi * x.dia / 7))
    df = df.assign(dia_cos = lambda x: np.cos(2 * np.pi * x.dia / 7))
    
    df = pd.concat((df, pd.get_dummies(df.mes, prefix = 'mes')), axis = 1)
    meses_presentes = df.mes.unique()
    if 1 not in meses_presentes:
        df = df.assign(mes_1 = 0)
    if 2 not in meses_presentes:
        df = df.assign(mes_2 = 0)
    if 3 not in meses_presentes:
        df = df.assign(mes_3 = 0)
    if 4 not in meses_presentes:
        df = df.assign(mes_4 = 0)
    if 5 not in meses_presentes:
        df = df.assign(mes_5 = 0)
    if 6 not in meses_presentes:
        df = df.assign(mes_6 = 0)
    if 7 not in meses_presentes:
        df = df.assign(mes_7 = 0)
    if 8 not in meses_presentes:
        df = df.assign(mes_8 = 0)
    if 9 not in meses_presentes:
        df = df.assign(mes_9 = 0)
    if 10 not in meses_presentes:
        df = df.assign(mes_10 = 0)
    if 11 not in meses_presentes:
        df = df.assign(mes_11 = 0)
    if 12 not in meses_presentes:
        df = df.assign(mes_12 = 0)
    df = df.assign(mes_sin = lambda x: np.sin(2 * np.pi * x.mes / 12))
    df = df.assign(mes_cos = lambda x: np.cos(2 * np.pi * x.mes / 12))

    return df[['cliente',
               'dia',
               'dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7',
               'dia_sin','dia_cos',
               'mes',
               'mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12',
               'mes_sin','mes_cos',
               'prod']]

def retirar_data_validacion(df):
    # No hay manera de saber qué registros pertenecen a los días 10 al 14 de mayo, así que se va a considerar todo mayo como validación
    return df[~((df.anho == '2021') & (df.mes == 'Mayo'))]

def preparar_dataset(df):
    df = df.drop(labels = ['_id','fecNacimiento','sexo','monto','descuento'], axis = 1)
    df = retirar_data_validacion(df)
    df = corregir_dtypes_dataframe(df)
    df = limpiar_dataset(df)
    df = generar_variables(df)
    return df
 
def generar_lista_variantes():
    variantes = {1: ['dia','prod'],
                 2: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','prod'],
                 3: ['dia_sin','dia_cos','prod'],
                 4: ['mes','prod'],
                 5: ['mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 6: ['mes_sin','mes_cos','prod'],
                 7: ['dia','mes','prod'],
                 8: ['dia','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 9: ['dia','mes_sin','mes_cos','prod'],
                 10: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes','prod'],
                 11: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 12: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes_sin','mes_cos','prod'],
                 13: ['dia_sin','dia_cos','mes','prod'],
                 14: ['dia_sin','dia_cos','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 15: ['dia_sin','dia_cos','mes_sin','mes_cos','prod']}
    return variantes
 
def aplicar_grid_search(X, y, modelo, params):
    if len(X) > 20:
        cv = KFold(10)
    else:
        cv = LeaveOneOut()
    gs = GridSearchCV(estimator = modelo,
                      param_grid = params,
                      cv = cv,
                      scoring = ['accuracy','f1_weighted'],
                      refit = 'f1_weighted',
                      return_train_score = True,
                      verbose = 2,
                      n_jobs = -1)
    gs.fit(X, y)
    return gs
 
def entrenar_LR(metodo_multiclass, X, y):
    print('algoritmo LR')
    modelo = LogisticRegression()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__C = np.logspace(-6,2,9), estimator__solver = ('newton-cg','sag','saga','lbfgs'))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_SVM(metodo_multiclass, X, y):
    print('algoritmo SVM')
    modelo = SVC()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__C = np.logspace(-6,2,9), estimator__gamma = np.logspace(-6,2,9))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_DT(metodo_multiclass, X, y):
    print('algoritmo DT')
    modelo = DecisionTreeClassifier()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__max_depth = range(2,11,2), estimator__min_samples_split = range(2,11,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_NB(metodo_multiclass, X, y):
    print('algoritmo NB')
    modelo = GaussianNB()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__var_smoothing = np.logspace(-11,-7,5))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_KNN(metodo_multiclass, X, y):
    print('algoritmo KNN')
    modelo = KNeighborsClassifier()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__n_neighbors = range(2, 31, 3), estimator__weights = ('uniform','distance'), estimator__p = (1,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar(df, variantes):
    global modelo_cliente
    lista_clientes = sorted(df.cliente.unique(), key = lambda x: int(x[3:]))
    lista_clientes.remove('SID1')
    modelo_cliente = list()
    for cliente in lista_clientes:
        print('cliente {}'.format(cliente))
        clasificadores = list()
        for cod_variante, variante in variantes.items():
            print('cod_variante {}'.format(cod_variante))
            base = df[df.cliente == cliente][variante].copy()
            X = base.drop(labels = 'prod', axis = 1)
            y = base['prod'].astype('int')
            if len(y.unique()) > 1:
                for metodo_multiclass in (OneVsOneClassifier, OneVsRestClassifier):
                    print('metodo_multiclass {}'.format(metodo_multiclass.__name__))
                    for entrenar_algoritmo in (entrenar_NB, entrenar_DT, entrenar_LR, entrenar_SVM):
                        clasificadores.append((cod_variante, entrenar_algoritmo(metodo_multiclass, X, y)))
            else:
                clasificadores.append((cod_variante, y.iloc[0]))
        modelo_cliente.append((cliente, clasificadores))
    return modelo_cliente
 
def main():
    df = crear_dataframe_desde_mongodb()
    df = preparar_dataset(df)
    variantes = generar_lista_variantes()
    c = entrenar(df, variantes)
    return c

modelo_cliente = None
 
np.random.seed(10)
ans = main()

cliente SID2
cod_variante 1
metodo_multiclass OneVsOneClassifier
algoritmo NB
Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  50 | elapsed:    4.2s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed:    6.4s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.2s finished


algoritmo DT
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   21.5s finished


algoritmo LR
Fitting 10 folds for each of 36 candidates, totalling 360 fits


...

[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 187 out of 250 | elapsed:    0.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.


algoritmo LR
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.


algoritmo SVM
Fitting 10 folds for each of 81 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:    0.6s


In [ ]:
import pickle
pickle_out = open('trabajo_v2.pkl', 'wb')
pickle.dump(ans, pickle_out)
pickle_out.close()

In [ ]:
!pip install scikit-learn==0.23.2 --upgrade

     |████████████████████████████████| 6.8MB 5.1MB/s 
     |████████████████████████████████| 307kB 34.7MB/s 
     |████████████████████████████████| 15.3MB 231kB/s 
     |████████████████████████████████| 27.4MB 96kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.

In [ ]:
import pickle
pickle_in = open('trabajo_v2.pkl', 'rb')
ans = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
from sklearn.model_selection import GridSearchCV
clientes_cod = {1:'SID2',2:'SID3',3:'SID5',4:'SID6',5:'SID7',6:'SID8',7:'SID9',8:'SID10',9:'SID12',10:'SID13',11:'SID14',12:'SID15',13:'SID17',14:'SID18',15:'SID20',16:'SID21',17:'SID22',18:'SID23',19:'SID24',20:'SID25',21:'SID26',22:'SID27',23:'SID28',24:'SID29',25:'SID30',26:'SID32',27:'SID33',28:'SID34',29:'SID35',30:'SID36',31:'SID38',32:'SID39',33:'SID40',34:'SID41',35:'SID42',36:'SID43',37:'SID44',38:'SID45',39:'SID46',40:'SID47',41:'SID48',42:'SID49',43:'SID50',44:'SID52',45:'SID53',46:'SID54',47:'SID55',48:'SID56',49:'SID57',50:'SID59',51:'SID61',52:'SID65',53:'SID66',54:'SID67',55:'SID68',56:'SID70',57:'SID71',58:'SID72',59:'SID73',60:'SID74',61:'SID75',62:'SID76',63:'SID77',64:'SID78',65:'SID79',66:'SID80',67:'SID81',68:'SID82',69:'SID83',70:'SID84',71:'SID85',72:'SID86',73:'SID87',74:'SID88',75:'SID89',76:'SID90',77:'SID91',78:'SID92',79:'SID93'}
n_features_variantes = {1: 1, 7: 2, 2: 3}
clasificadores = dict()
for cod_cliente, cliente_gs in enumerate(ans):
    print(clientes_cod[cod_cliente + 1])
    if type(cliente_gs[0]) == GridSearchCV:
        mejor_gs = sorted(cliente_gs, key = lambda gs: (gs.best_score_, gs.cv_results_['mean_test_accuracy'][gs.best_index_], gs.cv_results_['mean_train_f1_weighted'][gs.best_index_], gs.cv_results_['mean_train_accuracy'][gs.best_index_]), reverse = True)[0]
        mejor_clasificador = mejor_gs.best_estimator_
        variante = mejor_gs
    else:
        mejor_clasificador = cliente_gs[0]
        variante = None
    clasificadores[clientes_cod[cod_cliente + 1]] = (variante, mejor_clasificador)
    

SID2
SID3
SID5
SID6
SID7
SID8
SID9
SID10
SID12
SID13
SID14
SID15
SID17
SID18
SID20
SID21
SID22
SID23
SID24
SID25
SID26
SID27
SID28
SID29
SID30
SID32
SID33
SID34
SID35
SID36
SID38
SID39
SID40
SID41
SID42
SID43
SID44
SID45
SID46
SID47
SID48
SID49
SID50
SID52
SID53
SID54
SID55
SID56
SID57
SID59
SID61
SID65
SID66
SID67
SID68
SID70
SID71
SID72
SID73
SID74
SID75
SID76
SID77
SID78
SID79
SID80
SID81
SID82
SID83
SID84
SID85
SID86
SID87
SID88
SID89
SID90
SID91
SID92
SID93


In [ ]:
clasificadores

{'SID10': (2,
  OneVsRestClassifier(estimator=LogisticRegression(C=1e-06, solver='sag'))),
 'SID12': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID13': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID14': (2,
  OneVsOneClassifier(estimator=LogisticRegression(C=1e-06, solver='sag'))),
 'SID15': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID17': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID18': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID2': (3,
  OneVsOneClassifier(estimator=LogisticRegression(C=100.0, solver='newton-cg'))),
 'SID20': (None, 205),
 'SID21': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID22': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID23': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID24': (1, OneVsOneClassifier(estimator=GaussianNB

In [ ]:
pickle_out = open('clasificadores_v2.pkl', 'wb')
pickle.dump(clasificadores, pickle_out)
pickle_out.close()

In [ ]:
from sklearn.metrics import f1_score

df = crear_dataframe_desde_mongodb()
df = df.drop(labels = ['_id','fecNacimiento','sexo','monto','descuento'], axis = 1)

train = df[~((df.anho == '2021') & (df.mes == 'Mayo'))]
train = train.drop(labels = ['anho'], axis = 1)
train = corregir_dtypes_dataframe(train)
train = limpiar_dataset(train)
train = generar_variables(train)

test = df[((df.anho == '2021') & (df.mes == 'Mayo'))]
test = test.drop(labels = ['anho'], axis = 1)
test = corregir_dtypes_dataframe(test)
test = limpiar_dataset(test)
test = generar_variables(test)

clasificadores = obtener_clasificadores()
variantes = generar_lista_variantes()

curvas_por_cliente = list()
for cliente, (cod_variante, clasificador) in clasificadores.items():
    train_cliente = train[train.cliente == cliente][variantes[cod_variante]]
    X_train_cliente = train_cliente.drop(labels = 'prod', axis = 1)
    y_train_cliente = train_cliente['prod'].astype('int')
    
    test_cliente = test[test.cliente == cliente][variantes[cod_variante]]
    X_test_cliente = test_cliente.drop(labels = 'prod', axis = 1)
    y_test_cliente = test_cliente['prod'].astype('int')
    
    cantidad_training_samples = len(X_train_cliente)
    curva_aprendizaje = list()
    for train_samples in range(cantidad_training_samples):
        clasificador.fit(X_train_cliente.head(train_samples + 1), y_train_cliente.head(train_samples + 1))
        y_pred = clasificador.predict(X_test_cliente)
        score = f1_score(y_test_cliente, y_pred, average = 'weighted')
        curva_aprendizaje.append((train_samples, score))
    curvas_por_cliente.append(curvas_por_cliente)